In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

from scipy.sparse import csr_matrix, coo_matrix
from implicit.nearest_neighbours import ItemItemRecommender, CosineRecommender, TFIDFRecommender
from implicit.evaluation import train_test_split

In [2]:
data = pd.read_csv('data/retail_train.csv')
data.head()

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39,364,-0.60,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.00,1631,1,0.0,0.0
2,2375,26984851472,1,1036325,1,0.99,364,-0.30,1631,1,0.0,0.0
3,2375,26984851472,1,1082185,1,1.21,364,0.00,1631,1,0.0,0.0
4,2375,26984851472,1,8160430,1,1.50,364,-0.39,1631,1,0.0,0.0


In [3]:
data['week_no'].nunique()

95

In [4]:
users, items, interactions = data.user_id.nunique(), data.item_id.nunique(), data.shape[0]

print('# users: ', users)
print('# items: ', items)
print('# interactions: ', interactions)

# users:  2499
# items:  89051
# interactions:  2396804


In [5]:
popularity = data.groupby('item_id')['sales_value'].sum().reset_index()
popularity.describe()

,item_id,sales_value
count,8.905100e+04,89051.000000
mean,5.115772e+06,83.458481
std,5.178973e+06,1628.715079
min,2.567100e+04,0.000000
25%,9.665830e+05,3.500000
50%,1.448516e+06,10.780000
75%,9.553042e+06,46.105000
max,1.802456e+07,467993.620000


In [6]:
test_size_weeks = 3
data_train = data[data['week_no'] < data['week_no'].max() - test_size_weeks]
data_test = data[data['week_no'] >= data['week_no'].max() - test_size_weeks]

In [7]:
data_train.shape[0], data_test.shape[0]

(2278490, 118314)

# 1. Бейзлайны

In [8]:
result = data_test.groupby('user_id')['item_id'].unique().reset_index()
result.columns = ['user_id', 'actual']
result.head(2)

,user_id,actual
0,1,"[821867, 834484, 856942, 865456, 889248, 90795..."
1,3,"[835476, 851057, 872021, 878302, 879948, 90963..."


In [9]:
test_users = result.shape[0]
new_test_users = len(set(data_test['user_id']) - set(data_train['user_id']))

print('В тестовом дата сете {} юзеров'.format(test_users))
print('В тестовом дата сете {} новых юзеров'.format(new_test_users))

В тестовом дата сете 2042 юзеров
В тестовом дата сете 0 новых юзеров


## 1.1 Random recommendation

In [10]:
def random_recommendation(items, n=5):
    """Random recommendation."""
    
    items = np.array(items)
    recs = np.random.choice(items, size=n, replace=False)
    
    return recs.tolist()

In [11]:
%%time

items = data_train.item_id.unique()

result['random_recommendation'] = result['user_id'].apply(lambda x: random_recommendation(items, n=5))

result.head(2)

Wall time: 3.4 s


,user_id,actual,random_recommendation
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[13944875, 13007111, 13004615, 8020113, 12695541]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[13008132, 570830, 82789, 1493325, 13944709]"


## 1.2 Popularity-based recommendation

In [12]:
def popularity_recommendation(data, n=5):
    """Top-n popular items."""
    
    popular = data.groupby('item_id')['sales_value'].sum().reset_index()
    popular.sort_values('sales_value', ascending=False, inplace=True)
    
    recs = popular.head(n).item_id
    
    return recs.tolist()

In [13]:
%%time

popular_recs = popularity_recommendation(data_train, n=5)

result['popular_recommendation'] = result['user_id'].apply(lambda x: popular_recs)
result.head(2)

Wall time: 120 ms


,user_id,actual,random_recommendation,popular_recommendation
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[13944875, 13007111, 13004615, 8020113, 12695541]","[6534178, 6533889, 1029743, 6534166, 1082185]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[13008132, 570830, 82789, 1493325, 13944709]","[6534178, 6533889, 1029743, 6534166, 1082185]"


## 1.3 Weighted random recommender

### Задание 1. Weighted Random Recommendation

Напишите код для случайных рекоммендаций, в которых вероятность рекомендовать товар прямо пропорциональна логарифму продаж
- Можно сэмплировать товары случайно, но пропорционально какому-либо весу
- Например, прямопропорционально популярности. Вес = log(sales_sum товара)

In [14]:
def weighted_random_recommendation(items_weights, n=5):
    """Random recommendation items weighted with sales_sum.
    
    Input
    -----
    items_weights: pd.DataFrame
        DataFrame with columns item_id, weight. Weight sum of all items = 1
    """
    
    items = np.array(items_weights['item_id'])
    recs = np.random.choice(items, size=n, replace=False, p=items_weights['sales_value'])
    
    return recs.tolist()

In [15]:
items_weights = data_train.groupby('item_id')['sales_value'].sum().reset_index()
# items_weights['sales_value'] = items_weights['sales_value'].apply(lambda x: np.log(x) if x != 0 else 0)
items_weights_sum = items_weights['sales_value'].sum()
items_weights_sum
items_weights['sales_value'] = items_weights['sales_value'].apply(lambda x: x / items_weights_sum)
items_weights.head(3)

,item_id,sales_value
0,25671,2.969296e-06
1,26081,1.403822e-07
2,26093,2.254623e-07


In [16]:
weighted_recommendations = weighted_random_recommendation(items_weights, n=5)

result['weighted_random_recommendation'] = result['user_id'].apply(lambda x: weighted_recommendations)
result.head(5)

,user_id,actual,random_recommendation,popular_recommendation,weighted_random_recommendation
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[13944875, 13007111, 13004615, 8020113, 12695541]","[6534178, 6533889, 1029743, 6534166, 1082185]","[831628, 13671693, 889731, 885697, 844165]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[13008132, 570830, 82789, 1493325, 13944709]","[6534178, 6533889, 1029743, 6534166, 1082185]","[831628, 13671693, 889731, 885697, 844165]"
2,6,"[920308, 926804, 946489, 1006718, 1017061, 107...","[6602368, 1306466, 7407873, 12731697, 1364972]","[6534178, 6533889, 1029743, 6534166, 1082185]","[831628, 13671693, 889731, 885697, 844165]"
3,7,"[840386, 889774, 898068, 909714, 929067, 95347...","[987628, 9487587, 866252, 836386, 10255240]","[6534178, 6533889, 1029743, 6534166, 1082185]","[831628, 13671693, 889731, 885697, 844165]"
4,8,"[835098, 872137, 910439, 924610, 992977, 10412...","[1004341, 968154, 13133712, 845538, 991959]","[6534178, 6533889, 1029743, 6534166, 1082185]","[831628, 13671693, 889731, 885697, 844165]"


In [17]:
def precision(recommended_list, bought_list):
    """Precision calculation."""
    
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)
    
    flags = np.isin(bought_list, recommended_list)
    return flags.sum() / len(recommended_list)


def precision_at_k(recommended_list, bought_list, k=5):
    """Precision at k top items."""
    
    return precision(recommended_list[:k], bought_list)

In [18]:
for name_col in result.columns[1:]:
    print(f"{round(result.apply(lambda row: precision_at_k(row[name_col], row['actual']), axis=1).mean(),4)}:{name_col}")

1.0:actual
0.0003:random_recommendation
0.1552:popular_recommendation
0.0192:weighted_random_recommendation


# 2. Детерменированные алгоритмы (determenistic algorithms)

## 2.1 Item-Item Recommender / ItemKNN

In [19]:
popularity = data_train.groupby('item_id')['quantity'].sum().reset_index()
popularity.rename(columns={'quantity': 'n_sold'}, inplace=True)

popularity.head()

,item_id,n_sold
0,25671,6
1,26081,1
2,26093,1
3,26190,1
4,26355,2


In [20]:
top_5000 = popularity.sort_values('n_sold', ascending=False).head(5000).item_id.tolist()

In [21]:
data_train.head(100)

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39,364,-0.60,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.00,1631,1,0.0,0.0
2,2375,26984851472,1,1036325,1,0.99,364,-0.30,1631,1,0.0,0.0
3,2375,26984851472,1,1082185,1,1.21,364,0.00,1631,1,0.0,0.0
4,2375,26984851472,1,8160430,1,1.50,364,-0.39,1631,1,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
95,1060,26985040735,1,9553288,1,8.49,315,0.00,1251,1,0.0,0.0
96,1351,26985052379,1,903230,1,0.99,447,-0.30,1955,1,0.0,0.0
97,744,26985165432,1,5978648,0,0.00,31582,0.00,1119,1,0.0,0.0
98,212,26985205886,1,822346,1,1.25,288,-0.34,1341,1,0.0,0.0


In [22]:
data_train.loc[ ~ data_train['item_id'].isin(top_5000), 'item_id'] = 6666
data_train.head(100)

C:\Users\Admin\anaconda3\lib\site-packages\pandas\core\indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39,364,-0.60,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.00,1631,1,0.0,0.0
2,2375,26984851472,1,1036325,1,0.99,364,-0.30,1631,1,0.0,0.0
3,2375,26984851472,1,1082185,1,1.21,364,0.00,1631,1,0.0,0.0
4,2375,26984851472,1,8160430,1,1.50,364,-0.39,1631,1,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
95,1060,26985040735,1,9553288,1,8.49,315,0.00,1251,1,0.0,0.0
96,1351,26985052379,1,903230,1,0.99,447,-0.30,1955,1,0.0,0.0
97,744,26985165432,1,6666,0,0.00,31582,0.00,1119,1,0.0,0.0
98,212,26985205886,1,822346,1,1.25,288,-0.34,1341,1,0.0,0.0


In [23]:
user_item_matrix = pd.pivot_table(data_train,
                                  index='user_id', columns='item_id',
                                  values='quantity',
                                  aggfunc='count',
                                  fill_value=0
                                 )

user_item_matrix[user_item_matrix > 0] = 1

user_item_matrix = user_item_matrix.astype(float)

sparse_user_item = csr_matrix(user_item_matrix)

In [24]:
user_item_matrix.shape

(2499, 5001)

In [25]:
user_item_matrix.sum().sum() / (user_item_matrix.shape[0] * user_item_matrix.shape[1]) * 100

5.33770796861036

In [26]:
userids = user_item_matrix.index.values
itemids = user_item_matrix.columns.values

matrix_userids = np.arange(len(userids))
matrix_itemids = np.arange(len(itemids))

id_to_itemid = dict(zip(matrix_itemids, itemids))
id_to_userid = dict(zip(matrix_userids, userids))

itemid_to_id = dict(zip(itemids, matrix_itemids))
userid_to_id = dict(zip(userids, matrix_userids))

In [27]:
userid_to_id[2]

1

# Fit

In [28]:
%%time

model = ItemItemRecommender(K=5, num_threads=4)

model.fit(csr_matrix(user_item_matrix).T.tocsr(),
          show_progress=True
         )

recs = model.recommend(userid=userid_to_id[2],
                       user_items=csr_matrix(user_item_matrix).tocsr(),
                       N=5,
                       filter_already_liked_items=False,
                       filter_items=None,
                       recalculate_user=True
                      )

  0%|          | 0/5001 [00:00<?, ?it/s]

Wall time: 763 ms


In [29]:
recs

[(0, 78679.0),
 (3408, 72173.0),
 (2149, 57995.0),
 (3947, 17272.0),
 (3587, 14417.0)]

In [30]:
[id_to_itemid[rec[0]] for rec in recs]

[6666, 1082185, 981760, 1127831, 1098066]

In [31]:
%%time

result['itemitem'] = result['user_id'].apply(lambda user_id: [
                                    id_to_itemid[rec[0]] for rec in model.recommend(userid=userid_to_id[user_id],
                                                                                   user_items=sparse_user_item,
                                                                                   N=5,
                                                                                   filter_already_liked_items=False,
                                                                                   filter_items=None,
                                                                                   recalculate_user=True
                                                                                   )])

Wall time: 48 ms


In [32]:
result.head(5)

,user_id,actual,random_recommendation,popular_recommendation,weighted_random_recommendation,itemitem
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[13944875, 13007111, 13004615, 8020113, 12695541]","[6534178, 6533889, 1029743, 6534166, 1082185]","[831628, 13671693, 889731, 885697, 844165]","[6666, 1082185, 981760, 1127831, 995242]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[13008132, 570830, 82789, 1493325, 13944709]","[6534178, 6533889, 1029743, 6534166, 1082185]","[831628, 13671693, 889731, 885697, 844165]","[6666, 1082185, 981760, 1098066, 995242]"
2,6,"[920308, 926804, 946489, 1006718, 1017061, 107...","[6602368, 1306466, 7407873, 12731697, 1364972]","[6534178, 6533889, 1029743, 6534166, 1082185]","[831628, 13671693, 889731, 885697, 844165]","[6666, 1082185, 981760, 1127831, 995242]"
3,7,"[840386, 889774, 898068, 909714, 929067, 95347...","[987628, 9487587, 866252, 836386, 10255240]","[6534178, 6533889, 1029743, 6534166, 1082185]","[831628, 13671693, 889731, 885697, 844165]","[6666, 1082185, 981760, 1127831, 995242]"
4,8,"[835098, 872137, 910439, 924610, 992977, 10412...","[1004341, 968154, 13133712, 845538, 991959]","[6534178, 6533889, 1029743, 6534166, 1082185]","[831628, 13671693, 889731, 885697, 844165]","[6666, 1082185, 981760, 1127831, 1098066]"


## 2.2 CosineRecommender

In [33]:
%%time

model = CosineRecommender(K=5, num_threads=4)

model.fit(csr_matrix(user_item_matrix).T.tocsr(),
          show_progress=True
         )

  0%|          | 0/5001 [00:00<?, ?it/s]

Wall time: 587 ms


In [34]:
%%time

result['cosine'] = result['user_id'].\
    apply(lambda x: [id_to_itemid[rec[0]] for rec in
                    model.recommend(userid=userid_to_id[x],
                                    user_items=sparse_user_item,
                                    N=5,
                                    filter_already_liked_items=False,
                                    filter_items=None,
                                    recalculate_user=True)])

Wall time: 62 ms


In [35]:
result.head(3)

,user_id,actual,random_recommendation,popular_recommendation,weighted_random_recommendation,itemitem,cosine
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[13944875, 13007111, 13004615, 8020113, 12695541]","[6534178, 6533889, 1029743, 6534166, 1082185]","[831628, 13671693, 889731, 885697, 844165]","[6666, 1082185, 981760, 1127831, 995242]","[1082185, 6666, 981760, 1127831, 1098066]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[13008132, 570830, 82789, 1493325, 13944709]","[6534178, 6533889, 1029743, 6534166, 1082185]","[831628, 13671693, 889731, 885697, 844165]","[6666, 1082185, 981760, 1098066, 995242]","[1082185, 1098066, 981760, 6666, 826249]"
2,6,"[920308, 926804, 946489, 1006718, 1017061, 107...","[6602368, 1306466, 7407873, 12731697, 1364972]","[6534178, 6533889, 1029743, 6534166, 1082185]","[831628, 13671693, 889731, 885697, 844165]","[6666, 1082185, 981760, 1127831, 995242]","[1082185, 6666, 981760, 1127831, 1098066]"


## 2.3 TF-IDF взвешивание и TFIDFRecommender

In [36]:
%%time
model = TFIDFRecommender(K=5, num_threads=4)

model.fit(csr_matrix(user_item_matrix).T.tocsr(),
          show_progress=True)

  0%|          | 0/5001 [00:00<?, ?it/s]

Wall time: 590 ms


In [37]:
%%time

result['tfidf'] = result['user_id'].\
    apply(lambda x: [id_to_itemid[rec[0]] for rec in 
                    model.recommend(userid=userid_to_id[x], 
                                    user_items=sparse_user_item,   # на вход user-item matrix
                                    N=5, 
                                    filter_already_liked_items=False, 
                                    filter_items=None, 
                                    recalculate_user=False)])

Wall time: 59 ms


In [38]:
result.head(3)

,user_id,actual,random_recommendation,popular_recommendation,weighted_random_recommendation,itemitem,cosine,tfidf
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[13944875, 13007111, 13004615, 8020113, 12695541]","[6534178, 6533889, 1029743, 6534166, 1082185]","[831628, 13671693, 889731, 885697, 844165]","[6666, 1082185, 981760, 1127831, 995242]","[1082185, 6666, 981760, 1127831, 1098066]","[1082185, 981760, 1127831, 6666, 1098066]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[13008132, 570830, 82789, 1493325, 13944709]","[6534178, 6533889, 1029743, 6534166, 1082185]","[831628, 13671693, 889731, 885697, 844165]","[6666, 1082185, 981760, 1098066, 995242]","[1082185, 1098066, 981760, 6666, 826249]","[1082185, 981760, 1098066, 826249, 6666]"
2,6,"[920308, 926804, 946489, 1006718, 1017061, 107...","[6602368, 1306466, 7407873, 12731697, 1364972]","[6534178, 6533889, 1029743, 6534166, 1082185]","[831628, 13671693, 889731, 885697, 844165]","[6666, 1082185, 981760, 1127831, 995242]","[1082185, 6666, 981760, 1127831, 1098066]","[1082185, 981760, 1127831, 6666, 878996]"


## 2.4 Trick

In [39]:
%%time

model = ItemItemRecommender(K=1, num_threads=4)

model.fit(csr_matrix(user_item_matrix).T.tocsr(),
          show_progress=True)

  0%|          | 0/5001 [00:00<?, ?it/s]

Wall time: 540 ms


In [40]:
%%time

result['own_purchases'] = result['user_id'].\
    apply(lambda x: [id_to_itemid[rec[0]] for rec in 
                    model.recommend(userid=userid_to_id[x], 
                                    user_items=sparse_user_item,   # на вход user-item matrix
                                    N=5, 
                                    filter_already_liked_items=False, 
                                    filter_items=[itemid_to_id[6666]], 
                                    recalculate_user=True)])

Wall time: 49.6 ms


# Metrics

In [41]:
for name_col in result.columns[1:]:
    print(f"{round(result.apply(lambda row: precision_at_k(row[name_col], row['actual']), axis=1).mean(),4)}:{name_col}")

1.0:actual
0.0003:random_recommendation
0.1552:popular_recommendation
0.0192:weighted_random_recommendation
0.1368:itemitem
0.1329:cosine
0.139:tfidf
0.2199:own_purchases


C:\Users\Admin\AppData\Local\Temp/ipykernel_2924/364235188.py:8: RuntimeWarning: invalid value encountered in long_scalars
  return flags.sum() / len(recommended_list)


## Задание 2. Улучшение бейзлайнов и ItemItem

- Попробуйте улучшить бейзлайны, считая случаный на топ-5000 товаров
- Попробуйте улучшить разные варианты ItemItemRecommender, выбирая число соседей $K$.

### ItemItemRecommender for 1 NN

In [42]:
%%time

model = ItemItemRecommender(K=1, num_threads=4)

model.fit(csr_matrix(user_item_matrix).T.tocsr(),
          show_progress=True
         )

  0%|          | 0/5001 [00:00<?, ?it/s]

Wall time: 536 ms


In [43]:
%%time

result['itemitem_1'] = result['user_id'].apply(lambda user_id: [
                                    id_to_itemid[rec[0]] for rec in model.recommend(userid=userid_to_id[user_id],
                                                                                   user_items=sparse_user_item,
                                                                                   N=5,
                                                                                   filter_already_liked_items=False,
                                                                                   filter_items=None,
                                                                                   recalculate_user=True
                                                                                   )])

Wall time: 36 ms


In [44]:
for name_col in result.columns[1:]:
    print(f"{round(result.apply(lambda row: precision_at_k(row[name_col], row['actual']), axis=1).mean(),4)}:{name_col}")

1.0:actual
0.0003:random_recommendation
0.1552:popular_recommendation
0.0192:weighted_random_recommendation
0.1368:itemitem
0.1329:cosine
0.139:tfidf
0.2199:own_purchases
0.1923:itemitem_1


C:\Users\Admin\AppData\Local\Temp/ipykernel_2924/364235188.py:8: RuntimeWarning: invalid value encountered in long_scalars
  return flags.sum() / len(recommended_list)


In [45]:
for name_col in result.columns[1:]:
    print(f"{round(result.apply(lambda row: precision_at_k(row[name_col], row['actual']), axis=1).mean(),4)}:{name_col}")

1.0:actual
0.0003:random_recommendation
0.1552:popular_recommendation
0.0192:weighted_random_recommendation
0.1368:itemitem
0.1329:cosine
0.139:tfidf
0.2199:own_purchases
0.1923:itemitem_1


C:\Users\Admin\AppData\Local\Temp/ipykernel_2924/364235188.py:8: RuntimeWarning: invalid value encountered in long_scalars
  return flags.sum() / len(recommended_list)


In [46]:
for name_col in result.columns[1:]:
    print(f"{round(result.apply(lambda row: precision_at_k(row[name_col], row['actual']), axis=1).mean(),4)}:{name_col}")

1.0:actual
0.0003:random_recommendation
0.1552:popular_recommendation
0.0192:weighted_random_recommendation
0.1368:itemitem
0.1329:cosine
0.139:tfidf
0.2199:own_purchases
0.1923:itemitem_1


C:\Users\Admin\AppData\Local\Temp/ipykernel_2924/364235188.py:8: RuntimeWarning: invalid value encountered in long_scalars
  return flags.sum() / len(recommended_list)


### Using sales_value instead of quantity

In [47]:
popularity = data_train.groupby('item_id')['sales_value'].sum().reset_index()
popularity.rename(columns={'sales_value': 'sold'}, inplace=True)

popularity.head()

,item_id,sold
0,6666,2997550.99
1,202291,81.59
2,397896,2862.41
3,420647,463.81
4,480014,863.41


In [48]:
top_5000 = popularity.sort_values('sold', ascending=False).head(5000).item_id.tolist()

In [49]:
data_train.head(100)

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39,364,-0.60,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.00,1631,1,0.0,0.0
2,2375,26984851472,1,1036325,1,0.99,364,-0.30,1631,1,0.0,0.0
3,2375,26984851472,1,1082185,1,1.21,364,0.00,1631,1,0.0,0.0
4,2375,26984851472,1,8160430,1,1.50,364,-0.39,1631,1,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
95,1060,26985040735,1,9553288,1,8.49,315,0.00,1251,1,0.0,0.0
96,1351,26985052379,1,903230,1,0.99,447,-0.30,1955,1,0.0,0.0
97,744,26985165432,1,6666,0,0.00,31582,0.00,1119,1,0.0,0.0
98,212,26985205886,1,822346,1,1.25,288,-0.34,1341,1,0.0,0.0


In [50]:
data_train.loc[ ~ data_train['item_id'].isin(top_5000), 'item_id'] = 6666
data_train.head(100)

C:\Users\Admin\anaconda3\lib\site-packages\pandas\core\indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39,364,-0.60,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.00,1631,1,0.0,0.0
2,2375,26984851472,1,1036325,1,0.99,364,-0.30,1631,1,0.0,0.0
3,2375,26984851472,1,1082185,1,1.21,364,0.00,1631,1,0.0,0.0
4,2375,26984851472,1,8160430,1,1.50,364,-0.39,1631,1,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
95,1060,26985040735,1,9553288,1,8.49,315,0.00,1251,1,0.0,0.0
96,1351,26985052379,1,903230,1,0.99,447,-0.30,1955,1,0.0,0.0
97,744,26985165432,1,6666,0,0.00,31582,0.00,1119,1,0.0,0.0
98,212,26985205886,1,822346,1,1.25,288,-0.34,1341,1,0.0,0.0


In [51]:
user_item_matrix = pd.pivot_table(data_train, 
                                  index='user_id', columns='item_id', 
                                  values='sales_value',
                                  aggfunc='sum', 
                                  fill_value=0
                                 )

# user_item_matrix[user_item_matrix > 0] = 1 # так как в итоге хотим предсказать 

user_item_matrix = user_item_matrix.astype(float) # необходимый тип матрицы для implicit

# переведем в формат sparse matrix
sparse_user_item = csr_matrix(user_item_matrix).tocsr()

In [52]:
userids = user_item_matrix.index.values
itemids = user_item_matrix.columns.values

matrix_userids = np.arange(len(userids))
matrix_itemids = np.arange(len(itemids))

id_to_itemid = dict(zip(matrix_itemids, itemids))
id_to_userid = dict(zip(matrix_userids, userids))

itemid_to_id = dict(zip(itemids, matrix_itemids))
userid_to_id = dict(zip(userids, matrix_userids))

In [53]:
%%time

model = ItemItemRecommender(K=3, num_threads=4) # K - кол-во билжайших соседей


model.fit(csr_matrix(user_item_matrix).T.tocsr(),  # На вход item-user matrix
          show_progress=True)

  0%|          | 0/5000 [00:00<?, ?it/s]

Wall time: 543 ms


In [54]:
%%time

result['itemitem_sv'] = result['user_id'].apply(lambda user_id: [
                                     id_to_itemid[rec[0]]  for rec in model.recommend(userid=userid_to_id[user_id], 
                                                user_items=sparse_user_item,   # на вход user-item matrix
                                                N=5, 
                                                filter_already_liked_items=False, 
                                                filter_items=None, 
                                                recalculate_user=True)
                                                             ])

Wall time: 39 ms


In [55]:
for name_col in result.columns[1:]:
    print(f"{round(result.apply(lambda row: precision_at_k(row[name_col], row['actual']), axis=1).mean(),4)}:{name_col}")

1.0:actual
0.0003:random_recommendation
0.1552:popular_recommendation
0.0192:weighted_random_recommendation
0.1368:itemitem
0.1329:cosine
0.139:tfidf
0.2199:own_purchases
0.1923:itemitem_1
0.1432:itemitem_sv


C:\Users\Admin\AppData\Local\Temp/ipykernel_2924/364235188.py:8: RuntimeWarning: invalid value encountered in long_scalars
  return flags.sum() / len(recommended_list)


### Popularity-based recommendation

In [56]:
def popularity_recommendation(data, n=5):
    """Топ-n популярных товаров on count quantity."""
    
    popular = data.groupby('item_id')['quantity'].count().reset_index()
    popular.sort_values('quantity', ascending=False, inplace=True)
    
    recs = popular.head(n).item_id
    
    return recs.tolist()

In [57]:
%%time

# Можно так делать, так как рекомендация не зависит от юзера
popular_recs = popularity_recommendation(data_train, n=5)

result['popular_recommendation_2'] = result['user_id'].apply(lambda x: popular_recs)
result.head(2)

Wall time: 64 ms


,user_id,actual,random_recommendation,popular_recommendation,weighted_random_recommendation,itemitem,cosine,tfidf,own_purchases,itemitem_1,itemitem_sv,popular_recommendation_2
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[13944875, 13007111, 13004615, 8020113, 12695541]","[6534178, 6533889, 1029743, 6534166, 1082185]","[831628, 13671693, 889731, 885697, 844165]","[6666, 1082185, 981760, 1127831, 995242]","[1082185, 6666, 981760, 1127831, 1098066]","[1082185, 981760, 1127831, 6666, 1098066]","[1082185, 995242, 1029743, 840361, 904360]","[6666, 1082185, 995242, 1029743, 840361]","[6666, 6534178, 6533889, 995242, 1029743]","[6666, 1082185, 6534178, 1029743, 995242]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[13008132, 570830, 82789, 1493325, 13944709]","[6534178, 6533889, 1029743, 6534166, 1082185]","[831628, 13671693, 889731, 885697, 844165]","[6666, 1082185, 981760, 1098066, 995242]","[1082185, 1098066, 981760, 6666, 826249]","[1082185, 981760, 1098066, 826249, 6666]","[1082185, 1098066, 6534178, 826249, 1127831]","[6666, 1082185, 1098066, 6534178, 826249]","[6666, 6534178, 6533889, 1029743, 1106523]","[6666, 1082185, 6534178, 1029743, 995242]"


In [58]:
for name_col in result.columns[1:]:
    print(f"{round(result.apply(lambda row: precision_at_k(row[name_col], row['actual']), axis=1).mean(),4)}:{name_col}")

1.0:actual
0.0003:random_recommendation
0.1552:popular_recommendation
0.0192:weighted_random_recommendation
0.1368:itemitem
0.1329:cosine
0.139:tfidf
0.2199:own_purchases
0.1923:itemitem_1


C:\Users\Admin\AppData\Local\Temp/ipykernel_2924/364235188.py:8: RuntimeWarning: invalid value encountered in long_scalars
  return flags.sum() / len(recommended_list)


0.1432:itemitem_sv
0.1796:popular_recommendation_2


In [59]:
data_train

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39,364,-0.60,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.00,1631,1,0.0,0.0
2,2375,26984851472,1,1036325,1,0.99,364,-0.30,1631,1,0.0,0.0
3,2375,26984851472,1,1082185,1,1.21,364,0.00,1631,1,0.0,0.0
4,2375,26984851472,1,8160430,1,1.50,364,-0.39,1631,1,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
2282320,222,41297772783,635,1120741,1,0.59,304,0.00,1716,91,0.0,0.0
2282321,462,41297773713,635,6666,1,1.99,304,0.00,2040,91,0.0,0.0
2282322,462,41297773713,635,995242,1,1.00,304,-0.89,2040,91,0.0,0.0
2282323,462,41297773713,635,10180324,1,3.00,304,-0.29,2040,91,0.0,0.0


In [60]:
data.groupby('item_id')['quantity'].count().reset_index()

,item_id,quantity
0,25671,3
1,26081,1
2,26093,1
3,26190,1
4,26355,1
...,...,...
89046,17991689,1
89047,17991691,1
89048,18000012,3
89049,18024155,1


In [61]:
data.groupby('item_id')['quantity'].sum().reset_index()

,item_id,quantity
0,25671,6
1,26081,1
2,26093,1
3,26190,1
4,26355,2
...,...,...
89046,17991689,1
89047,17991691,1
89048,18000012,4
89049,18024155,1


## Косинусное сходство и CosineRecommender

#### Cosine for 1 NN

In [62]:
%%time

model = CosineRecommender(K=1, num_threads=4)

model.fit(csr_matrix(user_item_matrix).T.tocsr(),
          show_progress=True
         )

  0%|          | 0/5000 [00:00<?, ?it/s]

Wall time: 571 ms


In [63]:
%%time

result['cosine_1'] = result['user_id'].\
    apply(lambda x: [id_to_itemid[rec[0]] for rec in
                    model.recommend(userid=userid_to_id[x],
                                    user_items=sparse_user_item,
                                    N=5,
                                    filter_already_liked_items=False,
                                    filter_items=None,
                                    recalculate_user=True)])

Wall time: 41 ms


In [64]:
for name_col in result.columns[1:]:
    print(f"{round(result.apply(lambda row: precision_at_k(row[name_col], row['actual']), axis=1).mean(),4)}:{name_col}")

1.0:actual
0.0003:random_recommendation
0.1552:popular_recommendation
0.0192:weighted_random_recommendation
0.1368:itemitem
0.1329:cosine
0.139:tfidf
0.2199:own_purchases
0.1923:itemitem_1


C:\Users\Admin\AppData\Local\Temp/ipykernel_2924/364235188.py:8: RuntimeWarning: invalid value encountered in long_scalars
  return flags.sum() / len(recommended_list)


0.1432:itemitem_sv
0.1796:popular_recommendation_2
0.265:cosine_1


Для ItemItem + косинусного ItemItem лучшие показатели метрики получались для K=1